In [169]:
import ezdxf
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster




In [173]:
## Version 0 --> 
## algorithm: 
## find all vertices (areas are discarded) in alignment layer
## group vertices based on distance, separation between marks > max_d > single alignment mark
## detect center of mark, create X (Y) flipped copy of mark and detect shift where most points coincide
## requirements: mark mainly (in terms of amounts of points) symmetric around two main axes
##               center of mark is multiple of accuracy (best if center is on integer µm)


folder = '/Users/Lstaqdev/Downloads/Testalignmentmarks/' #folder for dxf in and dat output

file = 'SAGHUBBARDreduced.dxf' #name of DXF

outputname = 'Marks' # name of the outputfile

accuracy=1 #in µm

max_d = 213 #max size of alignment mark in µm

layers=['ebeam','outerebam'] #Layer(s) containining alingment marks



doc = ezdxf.readfile(folder+file)

msp = doc.modelspace()

points=[]

polylines = msp.query('POLYLINE')

for polyline in polylines:
    if polyline.dxf.layer in layers:
        for i, location in enumerate(polyline.points()):
            #print('Point at index {}: {}'.format(i, location))
            points.append([location[0],location[1]])

pointsnp = np.array(points)



Z = linkage(pointsnp,method='complete',metric='euclidean')                           


                 
clusters = fcluster(Z, max_d, criterion='distance')


allclusters=[]
for i in range(1,np.max(clusters)+1):
    clusterslist=[]
    for j in range(len(clusters)):
        
        if clusters[j]==i:
            clusterslist.append(points[j])
    allclusters.append(clusterslist)
    
allclustersnp = np.array(allclusters)

alignmentmarks = []
for i in allclustersnp:
    almark = mark(np.array(i))
    alignmentmarks.append(almark.get_center())
    
print(alignmentmarks)

np.savetxt(folder+outputname+'.dat', alignmentmarks)

C:\Users\Lstaqdev\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
C:\Users\Lstaqdev\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


[[75.0, 4925.0], [500.0, 4500.0], [750.0, 4250.0], [75.0, 75.0], [500.0, 500.0], [750.0, 750.0], [4925.0, 4925.0], [4500.0, 4500.0], [4250.0, 4250.0], [4925.0, 75.0], [4500.0, 500.0], [4250.0, 750.0]]


In [168]:
class mark:
    def __init__(self, clusterpoints,accuracy=1):
        self.__clusterpoints = clusterpoints
        self.accuracy = accuracy

    def __findextremaX(self):
        return(self.__clusterpoints[:,0].max(),self.__clusterpoints[:,0].min())
    
    def __findextremaY(self):
        return(self.__clusterpoints[:,1].max(),self.__clusterpoints[:,1].min())
    
    def __flipX(self,symaxisX):
        X = 2*symaxisX - self.__clusterpoints[:,0]
        Y = self.__clusterpoints[:,1]
        XY=np.zeros_like(self.__clusterpoints)
        XY[:,0]=X
        XY[:,1]=Y
        return(XY)
        
    def __flipY(self,symaxisY):
        X = self.__clusterpoints[:,0]
        Y = 2*symaxisY - self.__clusterpoints[:,1]
        XY=np.zeros_like(self.__clusterpoints)
        XY[:,0]=X
        XY[:,1]=Y
        return(XY)
    
    def __isin(self,X,Y,XY):
        for i in XY:
            if X==i[0] and Y==i[1]:
                return True
        return False
    
    def get_centerX(self):
        Xstart=int(self.__findextremaX()[1])
        Xend=int(self.__findextremaX()[0])
        steps=abs((Xend-Xstart)/self.accuracy) +1
        score=np.zeros(int(steps))
        for j,x in enumerate(np.linspace(Xstart,Xend,steps)): 
            mirror = self.__flipX(x)
            currentscore=0
            for i in range(np.shape(mirror)[0]):
                if self.__isin(mirror[i,0],mirror[i,1],self.__clusterpoints)== True:
                    currentscore+=1#*abs(mirror[i,0]-x)
                    #print('hit')
                    
            if currentscore>np.max(score):
                
                center=x
            score[j]=currentscore
            
        return center
       
    def get_centerY(self):
        Ystart=int(self.__findextremaY()[1])
        Yend=int(self.__findextremaY()[0])
        steps=abs((Yend-Ystart)/self.accuracy) +1
        score=np.zeros(int(steps))
        for j,y in enumerate(np.linspace(Ystart,Yend,steps)): 
            mirror = self.__flipY(y)
            currentscore=0
            for i in range(np.shape(mirror)[0]):
                if self.__isin(mirror[i,0],mirror[i,1],self.__clusterpoints)== True:
                    currentscore+=1#*abs(mirror[i,0]-x)
                    #print('hit')
                    
            if currentscore>np.max(score):
                
                center=y
            score[j]=currentscore
            
        return center
    
    def get_center(self):
        return [self.get_centerX(),self.get_centerY()]